In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
import pickle
import numpy as np
from keras.preprocessing import image, sequence

Using TensorFlow backend.


In [0]:
train_data = pd.read_csv('/content/gdrive/My Drive/Image Captioning Data/TrainImages.txt', delimiter='\t')
val_data = pd.read_csv('/content/gdrive/My Drive/Image Captioning Data/ValidationImages.txt', delimiter='\t')
train_sents_data = train_data.values
val_sents_data = val_data.values

In [0]:
train_sents = []
val_sents = []
for ix in range(train_sents_data.shape[0]):
  train_sents.append(train_sents_data[ix,1])

for ix in range(val_sents_data.shape[0]):
  val_sents.append(val_sents_data[ix,1])


In [0]:
print(len(train_sents))

30000


In [0]:
words = [i.split() for i in train_sents]
words.extend([i.split() for i in val_sents])

In [0]:
unique = []
for ix in range(len(words)):
  unique.extend(words[ix])

In [0]:
unique = list(set(unique))

In [0]:
words_to_indices = {word:idx for idx,word in enumerate(unique)}
indices_to_words = {idx:word for idx,word in enumerate(unique)}

In [0]:
total_words = len(unique)
word_0idx = indices_to_words[0]
indices_to_words[0] = 'UNK'
words_to_indices['UNK'] = 0
words_to_indices[word_0idx] = total_words
indices_to_words[total_words] = word_0idx
print(total_words, len(words_to_indices.keys()))

8918 8919


In [0]:
maxLen = 0
for ix in range(len(words)):
  maxLen = max(maxLen, len(words[ix]))

In [0]:
padded_seqs, subsequent_seqs = [] ,[]

for ix in range(train_sents_data.shape[0]):  ##Do Same for Validation.
  text = train_sents_data[ix,1].split()
  text = [words_to_indices[i] for i in text]
  next_words = text[1:]
  seq = text[:-1]
  padded_seqs.append(seq)
  subsequent_seqs.append(next_words)
captions = sequence.pad_sequences(padded_seqs, maxLen, padding='post')
next_words = sequence.pad_sequences(subsequent_seqs, maxLen, padding='post')
next_words = next_words.reshape((next_words.shape[0],next_words.shape[1],1))

In [0]:
print(captions.shape, next_words.shape)

(30000, 40) (30000, 40, 1)


In [0]:
with open('/content/gdrive/My Drive/Image Captioning Data/train_embs.pickle', 'rb') as file:
  encoded_images = pickle.load(file)

In [0]:
imgs = []
for ix in range(train_sents_data.shape[0]):
  img = encoded_images[train_sents_data[ix,0]]
  img = img.reshape((2048,))
  imgs.append(img)

In [0]:
imgs = np.array(imgs)
print(imgs.shape)

(30000, 2048)


In [0]:
np.save("./captions_train.npy", captions)       ##Do all same for validation
np.save("./next_words_train.npy", next_words)
np.save("./images_train.npy", imgs)